# Data Preprocessing
---

## Data Preprocessing Tools


### Import Libraries

In [9]:
import numpy as np # for converting or minipulating the data
import matplotlib.pyplot as plt # for plotting the data on graph to see
import pandas as pd # for importing the data

There are more tools that we will use as we advance.\
These are the basic tools for now so we can focus more understanding what we are doing and why.\
In fact there are libraries that we will use to help in the preprocessing stage.\
More on that later.

### Import Dataset
Here we imported that Data.csv and stored it in Variable called dataset.\
What csv_read returns is called a dataframe.\
A Dataframe is a 2 dimensional data structure, like a 2 dimensional array.

In [10]:
dataset = pd.read_csv('Data.csv')

In any dataset, that you will train a model with, you will have two disticnt entities.\
The set a features, which are the known characteristics of the observation.\
Or the independent variable(s).\
Then the set of dependent variable.\
\
Lets take a look at our data set.

In [11]:
print(dataset)

   Country   Age   Salary Purchased
0   France  44.0  72000.0        No
1    Spain  27.0  48000.0       Yes
2  Germany  30.0  54000.0        No
3    Spain  38.0  61000.0        No
4  Germany  40.0      NaN       Yes
5   France  35.0  58000.0       Yes
6    Spain   NaN  52000.0        No
7   France  48.0  79000.0       Yes
8  Germany  50.0  83000.0        No
9   France  37.0  67000.0       Yes


From here 4 colums, location, age, income, and if the bought a product.\
Real quick we can make out that if a person purchased a product is less likey to affect the other three columns.\
While any of the other may affect if someone does buy that product.\
We can safely say that (Country, Age, Salary) are the features and (Purchased) is the independent variable.\
We also have two cells with missing data.  More on that later.

In [15]:
# NOTE .values turns the dataframe to a numpy array
x = dataset.iloc[:,:-1].values # All Rows, All Columns except the last / feature set / independent variables
y = dataset.iloc[:,-1].values # All Rows, Only last column / output vecter / dependent variable

As already mentioned, dataset is a pandas dataframe that represents a tabel of data.\
Its not an array, but for now lets think of it as an array of arrays.\
We know in Python we index in a range like so: arr[ start : end : step ]\
We also know [ -1 ] is the same as indexing the last element.\
Well, what we might not know is that we can't index a dataframe like an array: dataset[ : ][ : -1 ]\
The Pandas dataframe frame has a built in method for indexing data called iloc.\
Pandas.iloc[  ] requires 1 argument but can take 2. indexing of the first dimension and then the second.\
Indexing values can bet both a range or a single index.

### Handle Missing Data
So as we seen above there was some data missing in our dataset.\
There a couple of ways to handle missing data:
- First if the dataset is very large and we are only missing a small %, we can just delete rows with missing data.
- A second way, and the way we'll do it is, replace the data with the avg of all the rows in that column of the dataset.

First we are add to out Data Preprocessing Tools.\
Normally we would import this with the rest of the imports at the top of the file.

In [16]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

Above we imported SimpleImputer is a class of sklearn library.\
Next we create a variable to store our new class obj and pass in a couple of arguments.\
First is missing_values, by pass np.nan we are basically saying and cell that doesnt have a value, or "not a number"
\
SimpleImputer also can do more than replace empty vaules with an average.\
You can also do things like the median, or most common value if its categorical like Country.\
\
NOTE: this set up is for numerical values, we will only want to apply it to the age and salary columns.\
For good practice, when doing this include all numerical columns, as we wont really know where there is missing data

In [17]:
imputer.fit(x[:,1:3])
x[:, 1:3] = imputer.transform(x[:,1:3])
print(x)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


We can see all the cells are full and the two oddballs are very obvious.

## Encoding Categorical Data
### Encoding The Independent Variable
Looking at our dataset, most of the data is a numerical value which is good.\
Our learning models will tend to have difficulty finding correlation with strings and the output vector.\
Which is why we are going to encode categorical data such as Country.\
You may think that we would just asign countries numerical values,\
but we dont want to give the impression that there is a ordering relation between countries.\
In other words Franch isn't first, Spain isn't second and so on.\
\
The Solution is "one hot encoding".\
This is were instead of giving a numerical value to cells with "Germany" we give a 1D vecter.\
So instead of it being:
- France = 0
- Spain = 1
- Germany = 2

it would be:
- France = [1,0,0]
- Spain = [0,1,0]
- Germany = [0,0,1]

In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
#                                say 2 encode, Encode Class, What Cols  // passthrough says to leave unencoded cols, default is to drop
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
x = np.array(ct.fit_transform(x)) # fit_transform returns a new matrix were all the countries are encoded, but is not a np array
# its easily converted us np.array()


### Encoding The Dependent Variable

## Split Data Into Train/Test Sets

---

## Feature Scaling